In [30]:
%load_ext autoreload
%autoreload plotmport numpy as np
import pandas as pd
import pandas_bokeh

import polygon_backfill as pb
import market_cluster as mc
import s3_datasets as s3d
import filters as ft
import local_backfill as lb

symbol='GLD'
date='2020-08-27'
result_path='/Users/bobcolner/QuantClarity/pandas-polygon/data'

# .plot_bokeh(kind='hist', bins=50, sizing_mode="scale_height", disable_scientific_axes=True)
# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
df = lb.load_ticks(symbol, date, result_path)

1  duplicated trade removed


In [32]:
df

,sequence,trade_id,exchange_id,price,size,condition,green,irregular,blank,afterhours,exchange_dt,sip_dt,filter,filter_diff,filter_pct,filter_zs
0,412401,52983525027889,11,182.990005,93,"[12, 37]",False,False,False,True,2020-08-27 08:00:00.010972160+00:00,2020-08-27 08:00:00.011008512+00:00,182.990005,0.000000,100.000000,-0.542293
1,412501,52983525027890,11,183.000000,1,"[12, 37]",False,False,False,True,2020-08-27 08:00:03.924153088+00:00,2020-08-27 08:00:03.924183040+00:00,182.995003,0.004997,99.997269,-0.144148
2,413101,52983525027891,11,182.970001,10,"[14, 12, 37, 41]",True,False,False,True,2020-08-27 08:01:53.674833664+00:00,2020-08-27 08:01:53.674867712+00:00,182.990005,0.020004,99.989067,1.051504
3,413701,52983525027892,11,182.869995,100,"[14, 12, 41]",True,False,False,True,2020-08-27 08:02:51.519661056+00:00,2020-08-27 08:02:51.519690496+00:00,182.980003,0.110008,99.939843,8.222379
4,413801,52983525027893,11,182.860001,1,"[14, 12, 37, 41]",True,False,False,True,2020-08-27 08:03:02.877552896+00:00,2020-08-27 08:03:02.877580544+00:00,182.970001,0.110001,99.939844,8.221771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124943,88027001,62880020642105,12,181.059998,100,"[14, 12, 41]",True,False,False,True,2020-08-27 23:56:48.145561649+00:00,2020-08-27 23:56:48.145902336+00:00,181.050003,0.009995,99.994480,0.253998
124944,88027101,52983527003702,1,181.070007,100,"[14, 12, 41]",True,False,False,True,2020-08-27 23:56:48.808652032+00:00,2020-08-27 23:56:48.808679680+00:00,181.059998,0.010010,99.994472,0.255213
124945,88027201,62880020642116,12,181.070007,100,"[14, 12, 41]",True,False,False,True,2020-08-27 23:56:48.808579034+00:00,2020-08-27 23:56:48.808920320+00:00,181.059998,0.010010,99.994472,0.255213
124946,88028301,79376134190665,4,181.100006,200,[12],False,False,False,True,2020-08-27 23:57:58.730000+00:00,2020-08-27 23:57:58.734042112+00:00,181.070007,0.029999,99.983435,1.847795


In [ ]:
# add price filters

df = ft.add_filters(df, 'price')

In [ ]:
df[['price', 'filter_jma710']][20000:30000].plot(figsize=[20,8])
# df[['price', 'filter_jma71000']][20000:30000].plot_bokeh(kind='line', sizing_mode="scale_height", disable_scientific_axes=True)

In [ ]:
df2 = df.rename(columns={'sip_dt': 'date_time', 'size': 'volume'})

df2 = df2[['date_time', 'price', 'volume']]

In [ ]:
# get data from s3
clean_df, bad_df = s3d.get_s3_df(symbol, date)

# list_s3_symbol('SPY')
df = s3d.get_s3_df('GLD', '2020-08-27', columns=['exchange_epoch', 'price', 'size', 'condition'])
# df = s3d.get_s3_df(symbol='SPY', date='2020-08-28', columns=['exchange_epoch', 'price', 'size', 'condition'])

# df = pd.read_feather('data.feather', columns=['exchange_epoch', 'price', 'size', 'condition'])
# df['date_time'] = pd.to_datetime(df.exchange_epoch)

In [ ]:
# get local market daily data
mdf = pd.read_parquet('data/mdf.parquet')

sdf = mdf[mdf.symbol=='SPY']

In [ ]:
### market clustering

# %time df = read_market_daily(result_path)
%time mdf = pd.read_parquet('data/mdf.parquet')

%time npdf, sharpe_ratios = mc.normalize_market_df(mdf)

# %time npdf_resid = colwise_linreg(npdf)
%time npdf_resid = pd.read_parquet('data/npdf_resid.parquet')

# %time par_cor_mat = npdf_resid.corr()
%time resid_par_cor_mat = pd.read_parquet('data/partial_cor_mat.parquet')

%time details_df = pd.read_parquet('data/details_df.parquet')

# cluster_lables = cluster_sim_matrix(similarity=resid_par_cor_mat.abs())
%time cluster_lables = pd.read_parquet('data/cluster_lables.parquet')

# symbol_meta = join_symbol_data(details_df, cluster_lables, sharpe_ratios, mdf)
%time symbol_meta = pd.read_parquet('data/symbol_meta.parquet')

# cluster_coheasion = mc.get_cluster_coheasion(sim_df=resid_par_cor_mat.abs(), symbol_meta=symbol_meta, cluster_col='cluster_n200')

In [ ]:
symbol_meta['liquidy_rank'] = symbol_meta.groupby('cluster_n200')['daily_avg_dollar_volume'].rank(method='first')

symbol_meta = symbol_meta.sort_values(['cluster_n200', 'liquidy_rank']).reset_index()

In [ ]:
by_cluster = symbol_meta.groupby('cluster_n200')
result = []
for cluster, frame in by_cluster:
    counts = dict(frame['sector'].value_counts())
    counts.pop('', None)  # drop symbols w/o sector/industry info
    result.append(counts)

cluster_sec = pd.DataFrame(result)    
cluster_sec = cluster_sec.drop(columns=['Finance'])
cluster_sec_pct = cluster_sec.apply(lambda x: 100 * x / float(x.sum()), axis=1)

In [ ]:
cluster_sec_pct.style.background_gradient(cmap='coolwarm', axis=0)

In [ ]:
# n += 1
n = 61
print(n)
symbol_meta[symbol_meta.cluster_n200==n].sector.value_counts()

In [ ]:
sym_clust = symbol_meta[symbol_meta.cluster_n200==n]
sym_clust